In [2]:
# downloads financial data (both current snapshot and time-series) via API using Yahoo Finance
import pandas as pd
import yfinance as yf
import datetime as datetime
import plotly.express as px

In [3]:
# downloads current snapshot (i.e. non-time series) via API

tickers = ['^GSPC', 'IEUR', 'MSFT', 'XOM'] # list tickers that you wanna download the data for
#tickers = ['^GSPC', 'XWD.TO', 'IEUR', 'EWU', '^HSI', 'VXUS', 'CL=F', 'GC=F']
fields_list = ['ticker', 'shortName', 'sector', 'industry', 'country', 'logo_url', 'floatShares', 'currency', 'financialCurrency', 'fullTimeEmployees', 'currentPrice', 'bid', 'ask', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow', 'totalRevenue', 'ebitda', 'netIncomeToCommon', 'totalDebt', 'totalCash', 'trailingEps', 'marketCap', 'enterpriseValue', 'enterpriseToEbitda', 'trailingPE', 'pegRatio', 'trailingPegRatio', 'dividendYield', 'fiveYearAvgDividendYield', 'payoutRatio', 'earningsGrowth', 'totalCashPerShare', 'ebitdaMargins', 'profitMargins', 'debtToEquity', 'recommendationMean', 'targetHighPrice', 'targetLowPrice', 'targetMedianPrice', 'targetMedianPrice', 'forwardPE', 'forwardEps', 'shortPercentOfFloat'] # creates a list of field names that I want to download the data for
dict_curr_data ={} # createsa an empty dictionary to host tickers as keys and API data values as DataFrames
df_curr_data = [] # deletes DataFrame in case it was created earlier

for ticker in tickers:
    ticker_obj = yf.Ticker(ticker) # downloads current data via API
    # converts info() output from dictionary to DataFrame
    temp_curr_data = pd.DataFrame.from_dict(ticker_obj.info, orient='index')
    temp_curr_data.reset_index(inplace=True)
    temp_curr_data.columns = ["Field", "Value"]
    dict_curr_data[ticker] = temp_curr_data # adds (ticker, dataframe) to the main dictionary

# combine this dictionary of dataframes into a single dataframe:
df_curr_data = pd.concat(dict_curr_data)
df_curr_data.reset_index(inplace=True)
df_curr_data.drop(['level_1'], axis=1, inplace=True) # deletes 'level_1' column
df_curr_data = df_curr_data[df_curr_data['Field'].isin(fields_list)] # deletes all rows where values under 'Field' are not listed in 'fields_list'
#df_curr_data

# for each ticker code add rows to calculate ND/Ebitda and Ebitda/employee
df_curr_data = df_curr_data.pivot(index='Field', columns='level_0', values='Value') # moves tickers from rows into columns
df_curr_data.loc['myEbitdaPerEmployee'] = df_curr_data.loc['ebitda'] / df_curr_data.loc['fullTimeEmployees'] # creates new row ('myEbitdaPerEmployee') using data from other existing rows ('ebitda' and 'fullTimeEmployees')
df_curr_data.loc['myNetDebtToEbitda'] = ( df_curr_data.loc['totalDebt'] - df_curr_data.loc['totalCash']) / df_curr_data.loc['ebitda']

# renames multiindex rows and turns them into single index
df_curr_data.reset_index(inplace=True) # convert 'Field' index to a standalone column and add a new rowindex
df_curr_data.index.names = ['ColIndex'] # change 'level_0' to 'ColIndex' as row-level index name
df_curr_data.columns.name = None # delete column-level index name
df_curr_data.reset_index(inplace=True) # convert 'ColIndex' index to a standalone column and add a new rowindex
df_curr_data.drop(['ColIndex'], axis=1, inplace=True) # delete 'ColIndex' index
df_curr_data

,Field,IEUR,MSFT,XOM,^GSPC
0,ask,48.4,250.06,104.92,3958.12
1,bid,48.39,250.08,104.93,3956.64
2,country,NaN,United States,United States,NaN
3,currency,USD,USD,USD,USD
4,currentPrice,NaN,250.3,105.21,NaN
5,debtToEquity,NaN,44.442,23.532,NaN
6,dividendYield,None,0.0111,0.0352,None
7,earningsGrowth,NaN,-0.133,1.981,NaN
8,ebitda,NaN,98841001984,82322997248,NaN
9,ebitdaMargins,NaN,0.48672,0.21141,NaN


In [4]:
# charts pricing data by importing OHLC data and saving it as 'df_px_close' DataFrame
df_px_close = [] # deletes DataFrame in case it was created earlier
df_px_close = pd.DataFrame() # creates empty DataFrame to store closing stock price data
for ticker in tickers: # loops through each ticker to download its closing share price and add it under a newe column of the DataFrame
    df_px_close[ticker] = yf.Ticker(ticker).history(period='5y').Close
    # another way to do it is here: https://www.allthesnippets.com/notes/finance/calculating_cumulative_returns_of_stocks_with_python_and_pandas.html

# charts prices of all tickers on a single graph
fig_px_close = px.line(df_px_close, 
                    x=df_px_close.index, 
                    y=df_px_close.columns,
                    height=450, 
                    width=750,
                    template='plotly_dark'
) # builds the chart
fig_px_close.show() # displays the chart

In [5]:
df_px_close

,^GSPC,IEUR,MSFT,XOM
Date,,,,
2017-12-12 00:00:00-05:00,2664.110107,42.867844,80.591965,63.633694
2017-12-13 00:00:00-05:00,2662.850098,42.927898,80.375359,63.910507
2017-12-14 00:00:00-05:00,2652.010010,42.696270,79.753830,63.741341
2017-12-15 00:00:00-05:00,2675.810059,42.610485,81.787941,63.841278
2017-12-18 00:00:00-05:00,2690.159912,43.193832,81.345337,63.772102
...,...,...,...,...
2022-12-06 00:00:00-05:00,3941.260010,48.279999,245.119995,103.879997
2022-12-07 00:00:00-05:00,3933.919922,48.270000,244.369995,103.650002
2022-12-08 00:00:00-05:00,3963.510010,48.439999,247.399994,104.419998


In [107]:
# charts cumulative return of all tickers in a single chart. Full instructions here: https://www.allthesnippets.com/notes/finance/calculating_cumulative_returns_of_stocks_with_python_and_pandas.html

# Calculates daily & cumulative returns
df_daily_returns = df_px_close.pct_change() # computes daily returns for each security
df_daily_returns = df_daily_returns[1:] # deletes first row with NaN values
df_cum_daily_returns = (1 + df_daily_returns).cumprod() - 1 # calculates cumulative daily returns
df_cum_daily_returns = df_cum_daily_returns.reset_index() # to add row_index to the DataFrame

# Transforms cumulative returns data for charting
df_px_cum_return = df_cum_daily_returns.melt(id_vars=['Date'], value_vars=tickers, var_name='Ticker', value_name='Cumulative Return')
df_px_cum_return['Cumulative Return Percent'] = df_px_cum_return['Cumulative Return'] * 100

# Charts cumulative returns
fig_px_cum_return = px.line(df_px_cum_return, x='Date',
              y='Cumulative Return Percent', color='Ticker',
              title='Performance: Daily Cumulative Returns',
              labels={'Cumulative Return Percent':'daily cumulative returns (%)', },
              height=450, 
              width=750,
              template='plotly_dark'
) # builds the chart
fig_px_cum_return.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
) # removes gridlines from the chart
fig_px_cum_return.show() # displays the chart

In [108]:
# calculates (cumulative?) return per security using formula: (last price - initial price) / initial price
cum_return = round(((df_px_close.iloc[-1] - df_px_close.iloc[0]) / df_px_close.iloc[0]) * 100)
cum_return

^GSPC     54.0
IEUR      15.0
MSFT     234.0
XOM       71.0
dtype: float64

In [ ]:
# # simple syntax to download current (non-time series) data using Yahoo Finance API

# tickers = 'MSFT'
# ticker_obj = yf.Ticker(tickers)
# stockinfo = ticker_obj.info

# # to view all entries in stockinfo dictionary as one line per item
# for key, value in stockinfo.items():
#     print(key, ":", value)